In [26]:
import pandas as pd
import numpy as np
import glob
import sys

places = pd.read_csv('/Users/adrua/Desktop/Fariy of Choice/archive_use/geoplaces2.csv')
ratings = pd.read_csv('/Users/adrua/Desktop/Fariy of Choice/archive_use/rating_final_new.csv')


print(places.shape)
print(ratings.shape)

(130, 21)
(1161, 6)


In [28]:
#user-item table로 변환
ratings = ratings[['userID', 'placeID', 'rating_total']]
ratings_matrix = ratings.pivot_table('rating_total', index='userID', columns='placeID')
ratings_matrix.head(3)

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
U1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN
U1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
#title 칼럼을 얻기 위해 places와 조인
ratings_places = pd.merge(ratings, places, on='placeID')

#columns='title'로 title 칼럼으로 피벗 수행
ratings_matrix = ratings_places.pivot_table('rating_total', index='userID', columns='name')

#NaN(평점 X) 값 모두 0 변환
ratings_matrix = ratings_matrix.fillna(0)

ratings_matrix.head(3)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El Rincón de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1003,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
#cosine_similarity
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Arrachela Grill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cabana Huasteca,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0


In [39]:
from sklearn.metrics.pairwise import cosine_similarity
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

#Cosine similarity로 반환된 넘파이 행렬을 영화명을 매핑해 Dataframe으로 변환
item_sim_df = pd.DataFrame(data=item_sim, 
                           index=ratings_matrix.columns, 
                           columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(129, 129)


name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El Rincón de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,1.0,0.0,0.0,0.000000,0.000000,0.300117,0.0,0.0,0.0,0.0,...,0.021918,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
Arrachela Grill,0.0,1.0,0.0,0.000000,0.408248,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
Cabana Huasteca,0.0,0.0,1.0,0.279706,0.000000,0.056474,0.0,0.0,0.0,0.0,...,0.469868,0.245572,0.044899,0.0,0.0,0.0,0.0,0.173507,0.0,0.0


In [42]:
item_sim_df["Abondance Restaurante Bar"].sort_values(ascending=False)[1:6]

name
Restaurante 75                      0.719310
la Estrella de Dimas                0.659223
Potzocalli                          0.615534
Restaurante y Pescaderia Tampico    0.581986
El Herradero Restaurante and Bar    0.465920
Name: Abondance Restaurante Bar, dtype: float64

In [44]:
item_sim_df["Arrachela Grill"].sort_values(ascending=False)[1:6]

name
El cotorreo             0.771517
Cafeteria cenidet       0.408248
Restaurant los Pinos    0.271607
El Oceano Dorado        0.215387
Mikasa                  0.154303
Name: Arrachela Grill, dtype: float64

In [47]:
#N의 범위에 제약을 두지 않는다면, 사용자별 영화 예측 평점 R_u,i는 사용자 u의 모든 영화에 대한 실제 평점과 
    #영화 i의 다른 모든 영화와의 코사인 유사도를 벡터 내적 곱(dot)한 값을 정규화를 위해 ***(p.620)로 나눈 것을 의미

def predict_rating(ratings_arr, item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [48]:
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, \
                                  columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El Rincón de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0.361132,0.0,0.456663,0.374036,0.0,0.637174,0.166974,0.0,0.0,0.083029,...,0.819871,0.286066,0.228808,0.124745,0.0,0.0,0.0,0.128771,0.0,0.0
U1002,0.144941,0.0,0.673209,0.492054,0.0,0.583358,0.550099,0.0,0.0,0.000000,...,0.911928,1.185455,0.315138,0.000000,0.0,0.0,0.0,0.459132,0.0,0.0
U1003,0.076171,0.0,2.034469,1.188339,0.0,0.345466,0.571342,0.0,0.0,0.000000,...,1.452990,3.812443,1.043952,0.000000,0.0,0.0,0.0,0.742500,0.0,0.0


In [49]:
from sklearn.metrics import mean_squared_error

#사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE를 구함 
def get_mse(pred, actual):
    #평점이 있는 영화만 추출
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 최근접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 최근접 이웃 MSE:  7.6547725697160045


In [67]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    #사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    
    pred = np.zeros(ratings_arr.shape)
    
    #열 크기만큼 루프 수행
    for col in range(ratings_arr.shape[1]):
        
        #유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 인덱스 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        
        #개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row,:][top_n_items].T)
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col,:][top_n_items]))
    
    return pred



In [53]:
#predict_rating_topsim()함수로 예측 평점 계산, 실제 평점과의 MSE를 구하기 & 예측 평점 넘파이 행렬은 판다스 Dataframe으로 재생성
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)
print('아이템 기반 최근접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values))

#예측 평점 데이터는 Dataframe으로 재생성 

ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)

아이템 기반 최근접 TOP-20 이웃 MSE:  6.143429476223462


/var/folders/sb/ggzbw3nx66b6nbvcr0yybr2w0000gn/T/ipykernel_22356/1830049633.py:14: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row,:][top_n_items].T)
/var/folders/sb/ggzbw3nx66b6nbvcr0yybr2w0000gn/T/ipykernel_22356/1830049633.py:15: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] /= np.sum(np.abs(item_sim_arr[col,:][top_n_items]))


In [77]:
user_rating_id = ratings_matrix.loc['U1001']
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

name
El Rincon de San Francisco          6.0
puesto de tacos                     5.0
Restaurante Versalles               4.0
Restaurant El Muladar de Calzada    3.0
Restaurant de Mariscos de Picon     3.0
Restaurant los Compadres            3.0
Restaurante la Gran Via             3.0
Rincon Huasteco                     3.0
Tortas Locas Hipocampo              2.0
Name: U1001, dtype: float64

In [85]:
def get_unvisit_places(ratings_matrix, userID):
    #userId로 입력받은 사용자의 모든 영화 정보를 추출해 Series로 반환
    #반환된 user_rating은 영화명(title)을 인덱스로 가지는 Series 객체
    user_rating = ratings_matrix.loc[userID, :]
    
    #user_rating이 0보다 크면 기존에 관람한 영화이다. 대상 인덱스를 추출해 list 객체로 만든다
    already_visit = user_rating[user_rating > 0].index.tolist()
    
    #모든 영화명을 list객체로 만듦
    places_list = ratings_matrix.columns.tolist()
    
    #list comprehension으로 already_seen에 해당하는 영화는 movies_list에서 제외함
    unvisit_places = [place for place in places_list if place not in already_visit]
    
    return unvisit_places

In [90]:
def recomm_place_by_userid(pred_df, userID, unvisit_list, top_n=10):
    #예측 평점 Dataframe에서 사용자id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출
    #가장 예측 평점이 높은 순으로 정렬
    recomm_places = pred_df.loc[userID, unvisit_list].sort_values(ascending=False)[:top_n]
    return recomm_places
    
#사용자가 관람하지 않는 영화명 추출
unvisit_list = get_unvisit_places(ratings_matrix, 'U1001')


#아이템 기반 최근접 이웃 협업 필터링으로 생성
recomm_places = recomm_place_by_userid(ratings_pred_matrix, 'U1001', unvisit_list, top_n=10)

recomm_places = pd.DataFrame(data = recomm_places.values, index = recomm_places.index, columns=['pred_score'])

recomm_places


,pred_score
name,
La Cantina Restaurante,1.107682
Restaurant Orizatlan,1.048755
Restaurante Casa de las Flores,1.033757
Unicols Pizza,0.995777
Preambulo Wifi Zone Cafe,0.958542
Restaurante El Cielo Potosino,0.875682
Restaurante Marisco Sam,0.866853
Restaurante Bar El Gallinero,0.855707
Restaurant Oriental Express,0.813340
